In [1]:
# Lo primero que hay que hacer es leer los datos del csv y guardarlos en una variable para trabajar con ellos.
# Despues hay que seleccionar que valores queremos utilizar como target y cuales como data.
# Una vez que los datos esten preparados 

import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR, NuSVR

In [2]:
peces = pd.read_csv('acuario.csv',sep=',')

X = peces[['Especie','Long_vert','Long_diag','Long_tras','Altura','Ancho']]
y = peces['Peso']

print(X.corr())

X = peces[['Especie','Long_vert','Altura','Ancho']]
# He quitado las longitudes transversales y diagonales porque estan correlacionadas a
# mas del 99% con la longitud vertical. Esto significa que dichos datos no son necesarios
# y eliminarlos mejoraria el rendimiento

            Especie  Long_vert  Long_diag  Long_tras    Altura     Ancho
Especie    1.000000  -0.217416  -0.230993  -0.262653 -0.483024 -0.391596
Long_vert -0.217416   1.000000   0.999517   0.992031  0.625378  0.867050
Long_diag -0.230993   0.999517   1.000000   0.994103  0.640441  0.873547
Long_tras -0.262653   0.992031   0.994103   1.000000  0.703409  0.878520
Altura    -0.483024   0.625378   0.640441   0.703409  1.000000  0.792881
Ancho     -0.391596   0.867050   0.873547   0.878520  0.792881  1.000000


REGRESION LINEAL

In [3]:
# Creamos el modelo de regresion. En este caso el lineal
modelo1 = linear_model.LinearRegression()

# Crear dos lista numpy vacias
scoreTrain=np.array([])
scoreTest=np.array([])

# Creamos un bucle de 100 iteraciones
for i in range(0,100):

    # En cada iteracion hacemos una division de los datos y entrenamos el modelos con esos valores
    # Stratify es muy importante porque hace que se cojan los datos en la proporcion que le pasemos en el train size:
        # El 50% de los peces de tipo 0, el 50% de los peces de tipo 1...
    XTrain, XTest, yTrain, yTest = train_test_split(X,y,stratify=X['Especie'], train_size=0.5)
    modelo1.fit(XTrain,yTrain)

    # Agregamos el score del train y test a sus respectivas listas
    scoreTest = np.append(scoreTrain,modelo1.score(XTrain,yTrain))
    scoreTrain = np.append(scoreTest,modelo1.score(XTest,yTest))

# Mostramos las medias de los scores obtenidos en las 100 iteraciones tanto para el test como para el train
print('Resultados del modelo de regresion lineal')
print('Score train:',scoreTrain.mean())
print('Score test:',scoreTest.mean())

Resultados del modelo de regresion lineal
Score train: 0.8780293870650836
Score test: 0.8783335058846512


REGRESION POLINOMICA

In [4]:
grado = [2,3,4,5]

for i in grado:
    modelo2 = linear_model.LinearRegression()
    pf = PolynomialFeatures(degree=i)

    # Crear dos lista numpy vacias
    scoreTrain=np.array([])
    scoreTest=np.array([])

    XPol = pf.fit_transform(X)

    # Creamos un bucle de 100 iteraciones
    for j in range(0,100):

        # En cada iteracion hacemos una division de los datos y entrenamos el modelos con esos valores
        XTrainPol, XTestPol, yTrain, yTest = train_test_split(XPol,y,stratify=X['Especie'],train_size=0.6)
        modelo2.fit(XTrainPol,yTrain)

        # Agregamos el score del train y test a sus respectivas listas
        scoreTest = np.append(scoreTrain,modelo2.score(XTrainPol,yTrain))
        scoreTrain = np.append(scoreTest,modelo2.score(XTestPol,yTest))

    # Mostramos las medias de los scores obtenidos en las 100 iteraciones tanto para el test como para el train
    print('Resultados del modelo de regresion polinomica de grado',i)
    print('Score train:',scoreTrain.mean())
    print('Score test:',scoreTest.mean(),'\n')

Resultados del modelo de regresion polinomica de grado 2
Score train: 0.9757281755600605
Score test: 0.9758690279024493 

Resultados del modelo de regresion polinomica de grado 3
Score train: 0.8468751009359019
Score test: 0.8464017249103306 

Resultados del modelo de regresion polinomica de grado 4
Score train: -57.48092404118788
Score test: -57.77290850109615 

Resultados del modelo de regresion polinomica de grado 5
Score train: -59867.755649411774
Score test: -59672.151107542144 



Queda claro viendo los valores obtenidos que el mejor grado para la regresion polinomica es el grado 2

MODELO SVR LINEAR (sin usar gridSearchCV)

In [5]:
#Introducimos los parametros para la creacion del modelo a mano.
modelo3 = SVR(C=1.0, epsilon=1, kernel='linear')

# Crear dos lista numpy vacias
scoreTrain=np.array([])
scoreTest=np.array([])

# Creamos un bucle de 100 iteraciones
for i in range(0,100):

    # En cada iteracion hacemos una division de los datos y entrenamos el modelos con esos valores
    XTrain, XTest, yTrain, yTest = train_test_split(X,y,stratify=X['Especie'],test_size=0.3)
    modelo3.fit(XTrain,yTrain)

    # Agregamos el score del train y test a sus respectivas listas
    scoreTrain = np.append(scoreTrain,modelo3.score(XTrain,yTrain))
    scoreTest = np.append(scoreTest,modelo3.score(XTest,yTest))

# Mostramos las medias de los scores obtenidos en las 100 iteraciones tanto para el test como para el train
print('Resultados del modelo de SVR con kernel linear')
print('Score train:',scoreTrain.mean())
print('Score test:',scoreTest.mean())

Resultados del modelo de SVR con kernel linear
Score train: 0.8705250289143666
Score test: 0.8528555411553124


MODELO SVR LINEAR (usando gridSearchCV).
GridSearchCV se encarga de elegir cuales son los mejores parametros posibles de todos los que le pases para el modelo que vas a entrenar.

In [6]:
#Elegimos los parametros que queremos optimizar y sus posibles valores
hiper={
    'epsilon':np.arange(1,101),
    'C':np.arange(1,10)
}

#Elegimos que tipo de modelo vamos a entrenar
modelo3 = SVR()

#Le pasamos al GridSearchCV el modelo, los hiperparametros y el cross validate
gscv = GridSearchCV(modelo3, hiper, cv=3)

#Entrenamos el modelo
gscv.fit(X,y)

# Obtenemos los mejores parametros
gscv.best_params_

{'C': 9, 'epsilon': 37}

In [7]:
#Creamos el modelo con los hiperparametros optimos calculados por el GridSearchCV
modelo3 = SVR(C=gscv.best_params_['C'], epsilon=gscv.best_params_['epsilon'], kernel='linear')

# Crear dos lista numpy vacias
scoreTrain=np.array([])
scoreTest=np.array([])

# Creamos un bucle de 100 iteraciones
for i in range(0,100):

    # En cada iteracion hacemos una division de los datos y entrenamos el modelos con esos valores
    XTrain, XTest, yTrain, yTest = train_test_split(X,y,stratify=X['Especie'],test_size=0.3)
    modelo3.fit(XTrain,yTrain)

    # Agregamos el score del train y test a sus respectivas listas
    scoreTrain = np.append(scoreTrain,modelo3.score(XTrain,yTrain))
    scoreTest = np.append(scoreTest,modelo3.score(XTest,yTest))

# Mostramos las medias de los scores obtenidos en las 100 iteraciones tanto para el test como para el train
print('Resultados del modelo de SVR con kernel linear usando los datos obtenidos por gridSearchCV')
print('Score train:',scoreTrain.mean())
print('Score test:',scoreTest.mean())

Resultados del modelo de SVR con kernel linear usando los datos obtenidos por gridSearchCV
Score train: 0.876468670572781
Score test: 0.8576515461693752


Al usar gridSearchCV podemos comprobar que los scores mejoran (en este caso ligeramente, pero mejoran) asi que a partir de ahora lo usaremos a la hora de entrenar todos los modelos.

MODELO SVR POLY (usando gridSearchCV)

In [8]:
#Elegimos los parametros que queremos optimizar y sus posibles valores
hiper={
    'epsilon':np.arange(1,11),
    'C':np.arange(1,11),
    'degree':np.arange(1,5)
}

#Elegimos que tipo de modelo vamos a entrenar
modelo4 = SVR()

#Le pasamos al GridSearchCV el modelo, los hiperparametros y el cross validate
gscv = GridSearchCV(modelo4, hiper, cv=3)

#Entrenamos el modelo
gscv.fit(X,y)

# Obtenemos los mejores parametros
gscv.best_params_

{'C': 10, 'degree': 1, 'epsilon': 10}

In [9]:
#Creamos el modelo con los hiperparametros optimos calculados por el GridSearchCV
modelo4 = SVR(C=gscv.best_params_['C'], epsilon=gscv.best_params_['epsilon'], kernel='poly', degree=gscv.best_params_['degree'])

# Crear dos lista numpy vacias
scoreTrain=np.array([])
scoreTest=np.array([])

# Creamos un bucle de 100 iteraciones
for i in range(0,100):

    # En cada iteracion hacemos una division de los datos y entrenamos el modelos con esos valores
    XTrain, XTest, yTrain, yTest = train_test_split(X,y,stratify=X['Especie'],test_size=0.3)
    modelo4.fit(XTrain,yTrain)

    # Agregamos el score del train y test a sus respectivas listas
    scoreTrain = np.append(scoreTrain,modelo4.score(XTrain,yTrain))
    scoreTest = np.append(scoreTest,modelo4.score(XTest,yTest))

# Mostramos las medias de los scores obtenidos en las 100 iteraciones tanto para el test como para el train
print('Resultados del modelo de SVR con kernel poly de grado',gscv.best_params_['degree'])
print('Score train:',scoreTrain.mean())
print('Score test:',scoreTest.mean())

Resultados del modelo de SVR con kernel poly de grado 1
Score train: 0.5683176277120243
Score test: 0.5633611296196762


MODELO SVR RBF (usando gridSearchCV)

In [10]:
#Elegimos los parametros que queremos optimizar y sus posibles valores
hiper={
    'epsilon':np.arange(1,11),
    'C':np.arange(1,100),
    'gamma':['scale','auto']
}

#Elegimos que tipo de modelo vamos a entrenar
modelo5 = SVR()

#Le pasamos al GridSearchCV el modelo, los hiperparametros y el cross validate
gscv = GridSearchCV(modelo5, hiper, cv=3)

#Entrenamos el modelo
gscv.fit(X,y)

# Obtenemos los mejores parametros
gscv.best_params_

{'C': 99, 'epsilon': 1, 'gamma': 'scale'}

In [11]:
#Creamos el modelo con los hiperparametros optimos calculados por el GridSearchCV
modelo5 = SVR(C=gscv.best_params_['C'], epsilon=gscv.best_params_['epsilon'], kernel='rbf', gamma=gscv.best_params_['gamma'])

# Crear dos lista numpy vacias
scoreTrain=np.array([])
scoreTest=np.array([])

# Creamos un bucle de 100 iteraciones
for i in range(0,100):

    # En cada iteracion hacemos una division de los datos y entrenamos el modelos con esos valores
    XTrain, XTest, yTrain, yTest = train_test_split(X,y,stratify=X['Especie'],test_size=0.3)
    modelo5.fit(XTrain,yTrain)

    # Agregamos el score del train y test a sus respectivas listas
    scoreTrain = np.append(scoreTrain,modelo5.score(XTrain,yTrain))
    scoreTest = np.append(scoreTest,modelo5.score(XTest,yTest))

# Mostramos las medias de los scores obtenidos en las 100 iteraciones tanto para el test como para el train
print('Resultados del modelo de SVR con kernel RBF')
print('Score train:',scoreTrain.mean())
print('Score test:',scoreTest.mean())

Resultados del modelo de SVR con kernel RBF
Score train: 0.8837899676181952
Score test: 0.8761319727626092


MODELO SVR SIGMOID (usando gridSearchCV)

In [12]:
#Elegimos los parametros que queremos optimizar y sus posibles valores
hiper={
    'epsilon':np.arange(1,11),
    'C':np.arange(1,100),
    'gamma':['scale','auto']
}

#Elegimos que tipo de modelo vamos a entrenar
modelo6 = SVR()

#Le pasamos al GridSearchCV el modelo, los hiperparametros y el cross validate
gscv = GridSearchCV(modelo6, hiper, cv=3)

#Entrenamos el modelo
gscv.fit(X,y)

# Obtenemos los mejores parametros
gscv.best_params_

{'C': 99, 'epsilon': 1, 'gamma': 'scale'}

In [13]:
#Creamos el modelo con los hiperparametros optimos calculados por el GridSearchCV
modelo6 = SVR(C=gscv.best_params_['C'], epsilon=gscv.best_params_['epsilon'], kernel='sigmoid', gamma=gscv.best_params_['gamma'])

# Crear dos lista numpy vacias
scoreTrain=np.array([])
scoreTest=np.array([])

# Creamos un bucle de 100 iteraciones
for i in range(0,100):

    # En cada iteracion hacemos una division de los datos y entrenamos el modelos con esos valores
    XTrain, XTest, yTrain, yTest = train_test_split(X,y,stratify=X['Especie'],test_size=0.3)
    modelo6.fit(XTrain,yTrain)

    # Agregamos el score del train y test a sus respectivas listas
    scoreTrain = np.append(scoreTrain,modelo6.score(XTrain,yTrain))
    scoreTest = np.append(scoreTest,modelo6.score(XTest,yTest))

# Mostramos las medias de los scores obtenidos en las 100 iteraciones tanto para el test como para el train
print('Resultados del modelo de SVR con kernel sigmoid')
print('Score train:',scoreTrain.mean())
print('Score test:',scoreTest.mean())

Resultados del modelo de SVR con kernel sigmoid
Score train: -3.0542435384035387
Score test: -3.1825941340891326


MODELO NUSVR RBF (usando gridSearchCV)

In [14]:
#Elegimos los parametros que queremos optimizar y sus posibles valores
hiper={
    'C':np.arange(1,100),
    'nu':[0.01,0.05,0.1,0.5,1,5],
    'gamma':['scale','auto'],
    'degree':np.arange(1,5)
}

#Elegimos que tipo de modelo vamos a entrenar
modelo7 = NuSVR()

#Le pasamos al GridSearchCV el modelo, los hiperparametros y el cross validate
gscv = GridSearchCV(modelo7, hiper, cv=3)

#Entrenamos el modelo
gscv.fit(X,y)

# Obtenemos los mejores parametros
gscv.best_params_

c:\Users\danie.LAPTOP-CLQ1QH4Q\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
2376 fits failed out of a total of 14256.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2376 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\danie.LAPTOP-CLQ1QH4Q\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\danie.LAPTOP-CLQ1QH4Q\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\danie.LAPTOP-CLQ1QH4Q\Ap

{'C': 99, 'degree': 1, 'gamma': 'scale', 'nu': 1}

In [15]:
#Creamos el modelo con los hiperparametros optimos calculados por el GridSearchCV
modelo7 = NuSVR(C=gscv.best_params_['C'], kernel='rbf', gamma=gscv.best_params_['gamma'], nu=gscv.best_params_['nu'], degree=gscv.best_params_['degree'])

# Crear dos lista numpy vacias
scoreTrain=np.array([])
scoreTest=np.array([])

# Creamos un bucle de 100 iteraciones
for i in range(0,100):

    # En cada iteracion hacemos una division de los datos y entrenamos el modelos con esos valores
    XTrain, XTest, yTrain, yTest = train_test_split(X,y,stratify=X['Especie'],test_size=0.3)
    modelo7.fit(XTrain,yTrain)

    # Agregamos el score del train y test a sus respectivas listas
    scoreTrain = np.append(scoreTrain,modelo7.score(XTrain,yTrain))
    scoreTest = np.append(scoreTest,modelo7.score(XTest,yTest))

# Mostramos las medias de los scores obtenidos en las 100 iteraciones tanto para el test como para el train
print('Resultados del modelo de NuSVR con kernel RBF de grado',gscv.best_params_['degree'])
print('Score train:',scoreTrain.mean())
print('Score test:',scoreTest.mean())

Resultados del modelo de NuSVR con kernel RBF de grado 1
Score train: 0.8831536645996588
Score test: 0.8703164362460319
